<a href="https://colab.research.google.com/github/chaeyoooo/capstondesign_voicefishing/blob/main/normaldata_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

normal_df = pd.read_csv("/content/drive/MyDrive/merged_normal.csv")

# 감정 분석 결과 로드
emotion_df = pd.read_csv("/content/drive/MyDrive/emotion_results.csv")

# 정상 데이터 + 감정분석 merge
merged_normal = pd.merge(normal_df, emotion_df, on='file', how='left')

# label 추가
merged_normal['label'] = 0

# 저장
merged_normal.to_csv("/content/drive/MyDrive/merged_normal.csv", index=False)

print(f"✅ merged_normal 저장 완료: {merged_normal.shape}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ merged_normal 저장 완료: (183, 11)


In [ ]:
import os
import json
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')


### ================================
### 1. 정상 데이터 json 폴더 경로
### ================================
normal_dir = "/content/drive/MyDrive/NIKL_DIALOGUE_2024_JSON_FILT"  # 폴더 경로 수정 필요

# json 파일 목록
json_files = [os.path.join(normal_dir, fname) for fname in os.listdir(normal_dir) if fname.endswith('.json')]

print(f"✅ json 파일 {len(json_files)}개 발견")

### ================================
### 2. json → transcribed_text 추출
### ================================
normal_texts = []
file_names = []

for file_path in json_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        utterances = []
        for doc in data.get('document', []):
            for utt in doc.get('utterance', []):
                utterances.append(utt.get('form', ''))
        # 각 파일에서 나온 모든 utterance → 하나의 text로 합치기
        text_concat = " ".join(utterances)
        normal_texts.append(text_concat)
        file_names.append(os.path.basename(file_path))

### ================================
### 3. DataFrame 생성
### ================================
normal_df = pd.DataFrame({
    'file': file_names,
    'transcribed_text': normal_texts
})

print(f"✅ 정상 데이터 DataFrame 생성 완료: {normal_df.shape[0]} rows")
print(normal_df.head())

### ================================
### 4. keyword flag 추가 (같은 keyword 리스트 사용)
### ================================
custom_keywords = [
    '통장', '고객', '은행', '계좌', '금융',
    '카드', '조사', '대출', '피해자', '연락',
    '자금', '피해', '수사', '진술', '불법'
]

def has_keyword(text):
    return int(any(keyword in text for keyword in custom_keywords))

normal_df['has_keyword'] = normal_df['transcribed_text'].apply(has_keyword)

print("✅ keyword flag 추가 완료")
print(normal_df[['file', 'transcribed_text', 'has_keyword']].head())

### ================================
### 5. 감정분석 결과 merge
### ================================
emotion_df = pd.read_csv("/content/drive/MyDrive/emotion_results.csv")

merged_normal = pd.merge(normal_df, emotion_df, on='file', how='left')

print("✅ 감정분석 merge 완료")
print(merged_normal[['file', 'transcribed_text', 'emotion', 'confidence', 'has_keyword']].head())

### ================================
### 6. is_deepfake 컬럼 추가
### ================================
merged_normal['is_deepfake'] = 0

print("✅ is_deepfake 컬럼 추가 완료")
print(merged_normal[['file', 'transcribed_text', 'emotion', 'confidence', 'has_keyword', 'is_deepfake']].head())

### ================================
### 7. 저장
### ================================
merged_normal.to_csv("/content/drive/MyDrive/merged_normal.csv", index=False)

print("✅ merged_normal 저장 완료 → /content/drive/MyDrive/merged_normal.csv")


Mounted at /content/drive
✅ json 파일 183개 발견
✅ 정상 데이터 DataFrame 생성 완료: 183 rows
                  file                                   transcribed_text
0  SDRW2400000051.json  저는 최근에 보면 이런 어 투자하고 있는 그런 게 있는데 보면 주식이나 이런 거에 ...
1  SDRW2400000107.json  저는 현재 경제 상황이 어 나쁘지도 좋다고도 생각을 안 하고 있어요. 근데 요즘 경...
2  SDRW2400000172.json  name1 님은 최근에 하고 계신 재테크나 관심 있는 금융 서비스 같은 거 있으세요...
3  SDRW2400000320.json  요즘 우리 나이 세대뿐만 아니고 젊은 분들이나 나이 많으신 분들이나 재테크 관련해 ...
4  SDRW2400000357.json  현재 거주 중인 그 부동산 장단점에 대해서 우리 한번 이야기해 볼까요? 그 아 우리...
✅ keyword flag 추가 완료
                  file                                   transcribed_text  \
0  SDRW2400000051.json  저는 최근에 보면 이런 어 투자하고 있는 그런 게 있는데 보면 주식이나 이런 거에 ...   
1  SDRW2400000107.json  저는 현재 경제 상황이 어 나쁘지도 좋다고도 생각을 안 하고 있어요. 근데 요즘 경...   
2  SDRW2400000172.json  name1 님은 최근에 하고 계신 재테크나 관심 있는 금융 서비스 같은 거 있으세요...   
3  SDRW2400000320.json  요즘 우리 나이 세대뿐만 아니고 젊은 분들이나 나이 많으신 분들이나 재테크 관련해 ...   
4  SDRW2400000357.json  현재 거주 중인 그 부동산 장단점에 대해서 우리 한번 이야기해 볼까요? 그 아 우리.